In [2]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Define the directory containing the PDF files
pdf_directory = './data'

# Use glob to get all PDF files in the directory
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)

In [5]:
# Split the documents into chunks
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

In [ ]:
# Create embeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

In [ ]:
# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# Define the chatbot response function
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

Read Json

In [19]:
import json

# Read the JSON file
with open('data/kgls_dummy.json') as file:
    kgls = json.load(file)

# Print the contents of the JSON file
print(kgls['Family'][0]['question'])
print(kgls['Family'][1]['question'])
print()
with open('data/qa.json') as file:
    q = json.load(file)
print(q['Q'][0]['Qusetion'])
print(q['Q'][1]['Qusetion'])
print(q['Q'][2]['Qusetion'])
print(q['A'][0]['Answer'])



가족과의 대화 시간이 부족하다고 느낀다.
가족과 함께 있는 시간이 충분하지 않다고 생각한다.

언제 외로움을 느끼셨나요? 예시를 들어주세요.
외롭다는 사실을 알게되신 상황이 어떤건가요?
가족과 있을때 필요한것이 무엇이었나요?
가장 외로웠던 상황은 가족들과 떨어져 살았을때 입니다.


In [3]:
from gpt_class import GPT

load_dotenv()
openai = os.getenv("OPENAI_API_KEY")
## Example
gpt = GPT(api_key=openai, model="gpt-4o", sys_prompt="너는 외로움을 분석하는 모델이야 사용자의 kgls결과를 바탕으로 사용자의 외로움을 분석하고 답변해줄 수 있어.")
img_prompt =[]


In [10]:
import json
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

dir = './data'

pdf_files = glob(os.path.join(dir, '*.pdf'))
json_files = glob(os.path.join(dir, '*.json'))
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)

# for json_file in json_files:
#     with open(json_file) as file:
#         jsons = json.load(file)
#     documents.append(jsons)
            
#청크로 나누고 문서를 저장
# RecursiveCharacterTextSplitter를 사용하여 문서를 청크로 나누기
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

#임베딩
#OpenAi와 Chormadb사용
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

#QA체인 만들기
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4", temperature=0),
    chain_type="stuff",
    
    retriever=retriever,
    return_source_documents=True
)

ValidationError: 1 validation error for RetrievalQA
prompt
  extra fields not permitted (type=value_error.extra)